<a href="https://colab.research.google.com/github/punchnami/AI-Powered-Research-Query-Chatbot/blob/main/RAG_GPT4mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install chromadb==0.5.3
!pip install langchain
!pip install unstructured
!pip install langchain_community
!pip install sentence-transformers
! pip install openai
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 123.0 MB/s eta 0:00:0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import chromadb
import torch
import os
import tensorflow as tf

os.environ['USER_AGENT'] ='myagent'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader


document_loader = DirectoryLoader('/content/drive/MyDrive/chatbot/datasources')
data = document_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

# embeddings = OpenAIEmbeddings()
# embeddings_gpt = embeddings.embed_documents([doc.page_content for doc in all_splits])

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
hf_embeddings_vectors = embeddings.embed_documents([doc.page_content for doc in all_splits])

chroma_client = chromadb.Client()

# Add to Vector database ChromaDB
# Vector ChromaDB's 0.5.4 current bug https://github.com/langchain-ai/langchain/issues/24163
# vectorstore = Chroma.from_documents(
#     documents=all_splits,
#     collection_name="rag-chroma-2",
#     embedding=embeddings,
# )

batch_size = 41666
split_batches = [all_splits[i:i + batch_size] for i in range(0, len(all_splits), batch_size)]

for i, batch in enumerate(split_batches):
    print(f"Adding batch {i + 1}/{len(split_batches)}")
    vectorstore = Chroma.from_documents(
        documents=batch,
        collection_name=f"rag-chroma-2-batch-{i}",
        embedding=embeddings,
        client=chroma_client
    )

print("collections after ", chroma_client.list_collections())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Adding batch 1/2
Adding batch 2/2
collections after  [<chromadb.api.models.Collection.Collection object at 0x78ee47778e80>, <chromadb.api.models.Collection.Collection object at 0x78ee477780a0>]


In [ ]:
import os
import json
import torch
import openai
from transformers import AutoTokenizer, AutoModelForCausalLM

path_to_API_key = "/content/drive/MyDrive/chatbot/gpt_api_key.json"
with open(path_to_API_key, "r") as file:
    api_key = json.load(file)
    os.environ["OPENAI_API_KEY"] = api_key["OPENAI_API_KEY"]
openai.api_key = os.environ["OPENAI_API_KEY"]

available_memory = torch.cuda.get_device_properties(0).total_memory if torch.cuda.is_available() else None

In [ ]:
from langchain.chat_models import ChatOpenAI

# Prompt
question_check_template = """
question: {question}
"""

QUESTION_CHECK_PROMPT = PromptTemplate(
    template=question_check_template, input_variables=["question"]
    )

openai_llm = ChatOpenAI(model='gpt-4o-mini')

# this is to check if there exist an answer for the given question from the vectorstore retriever
retriever = vectorstore.as_retriever()
qa_check = RetrievalQA.from_chain_type(llm=openai_llm, chain_type=
                                       "stuff", retriever= vectorstore.as_retriever())
qa_check.combine_documents_chain.llm_chain.prompt = QUESTION_CHECK_PROMPT

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
!pip install gradio

In [ ]:
def query_qa_check(question):
    return qa_check.invoke({"query":question})

# Example query
response = query_qa_check("What factors influence the determination of cash outflow rates in OSFI's LCR framework?")
print(response['result'])

In the Office of the Superintendent of Financial Institutions (OSFI) Liquidity Coverage Ratio (LCR) framework, several factors influence the determination of cash outflow rates. These factors include:

1. **Deposit Type**: Different types of deposits have varying levels of stability. For instance, retail deposits may be considered less volatile compared to wholesale funding, leading to different outflow rates.

2. **Customer Behavior**: Historical withdrawal patterns and the behavior of different customer segments can influence outflow estimates. For example, business customers may have different cash management practices than individual consumers.

3. **Market Conditions**: Economic conditions and market liquidity can affect how quickly depositors might withdraw funds during times of stress. In turbulent markets, outflow rates may be adjusted to reflect increased withdrawal tendencies.

4. **Institutional Factors**: The specific characteristics of the financial institution, such as it

In [ ]:
import gradio as gr

def chatbot_response(user_input):
    response = qa_check.invoke({"query": user_input})
    response = response["result"].split("answer:")[-1]
    return response

iface = gr.Interface(
    fn=chatbot_response,
    inputs="text",
    outputs="text",
    title="Chatbot",
    description="Ask me anything!",
)

iface.launch()
#iface.launch(debug=True)

In [ ]:
!pip install scikit-learn sentence-transformers

In [ ]:
# Cosine similarities with the documents
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_cosine_similarity(query_embedding, document_embeddings):
    return cosine_similarity([query_embedding], document_embeddings)

queries = ["What specific characteristics must an asset have to be eligible as High-Quality Liquid Assets under OSFI guidelines?",
           "What factors influence the determination of cash outflow rates in OSFI's LCR framework?",
           "How does OSFI assess the impact of market stress on the liquidity of assets included in HQLA?",
           "How should institutions manage liquidity risk arising from currency mismatches as per OSFI guidelines?",
           "What does OSFI require regarding documentation and legal opinions on intra-group liquidity transfers?",
           "What are OSFI's expectations for financial institutions' liquidity stress testing practices and scenarios?",
           "What are the reporting frequencies and deadlines for LCR compliance as mandated by OSFI?",
           "What is the ASF factor associated to: Total regulatory capital (excluding Tier 2 instruments with residual maturity of less than one year)? ",
           "Stable non-maturity (demand) deposits and term deposits with residual maturity of less than one year provided by retail and small business customers? ",
           "All less stable non-maturity deposits and term deposits with residual maturity of less than one year provided by retail and small business customers not assigned a lower ASF factor? ",
           "Rate sensitive deposits managed by the client, no relationship and deposit not in a transactional account? ",
           "Term deposits directly managed by an unaffiliated third party (maturing or cashable in the next 30 days)? ",
           "What is the ASF factor associated to	Demand deposits directly managed by an unaffiliated third party? ",
           "What is the ASF factor associated to Funding with residual maturity of less than one year provided by non-financial corporate customers? ",
           "Funding with residual maturity of less than one year from sovereigns, PSEs, and multilateral and national development banks? ",
           "	Stamped bankers' acceptances (BA) liabilities issued by the institution with a residual maturity of less than six months? ",
           "	Matched secured financing transactions that meet the criteria for matched transactions outlined in paragraph 39? ",
           "	NSFR derivative liabilities net of NSFR derivative assets if NSFR derivative liabilities are greater than NSFR derivative assets? ",
           "Trade date payables arising from purchases of financial instruments, foreign currencies and commodities? ",
           "What is the RSF factor associated to: Coins and banknotes?",
           "What is the RSF factor associated to:	All claims on central banks with residual maturities of less than six months?",
           "What is the RSF factor associated to: Assets associated with collateral posted as variation margin that are deducted from the replacement cost of derivative liability amounts?",
           "What is the RSF factor associated to:Unencumbered loans to financial institutions with residual maturities of less than six months, where the loan is secured against Level 1 and where the institution has the ability to freely rehypothecate the received collateral for the life of the loan?",
           "What is the RSF factor associated to:Unencumbered loans to financial institutions with residual maturities of less than six months, where the loan is secured against non-Level 1 assets, and where the institution has the ability to freely rehypothecate the received collateral for the life of the loan?",
           "What is the RSF factor associated to:	All other unencumbered loans to financial institutions with residual maturities of less than six months not included in the above categories?",
           "What is the RSF factor associated to:	Unencumbered Level 2A assets?",
           "What is the RSF factor associated to:	HQLA encumbered for a period of six months or more and less than one year?",
           "What is the RSF factor associated to:	Loans to financial institutions and central banks with residual maturities between six months and less than one year? ",
           "What is the RSF factor associated to:	Deposits held at other financial institutions for operational purposes? ",
           "What is the RSF factor associated to: 	All other assets not included in the above categories with residual maturity of less than one year, including loans to non-financial corporate clients, loans to retail and small business customers, loans to sovereigns and PSEs, and loans to national development banks? ",
           "What is the RSF factor associated to:	Unencumbered residential mortgages with a residual maturity of one year or more and with a risk weight of less than or equal to 35% under the Standardised Approach for credit risk?",
           "What is the RSF factor associated to: 	Unencumbered reverse mortgages that would qualify for a 35% risk weight under the Standardized Approach for credit risk? ",
           "What is the RSF factor associated to: 	Cash, securities or other assets posted as initial margin for derivative contracts and cash or other assets provided to contribute to the default fund of a CCP?",
           "What is the RSF factor associated to:	Unencumbered reverse mortgages that would qualify for a 50%, 75%, or 100% risk weight under the Standardized Approach for credit risk?",
           "What is the RSF factor associated to: 	All assets that are encumbered for a period of one year or more?",
           "What is the RSF factor associated to:	NSFR derivative assets net of NSFR derivative liabilities if NSFR derivative assets are greater than NSFR derivative liabilities?",
           "What is the RSF factor associated to:	5% of derivative liabilities (i.e. negative replacement cost amount) calculated before deducting variation margin posted?",
           "What is the outflow rate applicable to Corporate Credit commitments?",
           "Can you provide a comparison of the outflow treatment for a retail less stable deposit between NCF and LCR?",
           ]


def query_qa_check(question):
    return qa_check.invoke({"query":question})

scores_docs_list = []
source_lists = []

for _ in range(5):
    scores_doc = {}
    source_list = []

    for query in queries:
        response = query_qa_check(query)['result']
        query_embedding = embeddings.embed_query(response)

        similarities = compute_cosine_similarity(query_embedding, hf_embeddings_vectors)
        most_similar_index = np.argmax(similarities)
        most_similar_doc = all_splits[most_similar_index]

        scores_doc[query] = similarities[0][most_similar_index]
        source_list.append(most_similar_doc.metadata.get('source', 'No source available'))

        # print("Most similar context from the document:", most_similar_doc.page_content)
        # print("Source of the most similar context:", most_similar_doc.metadata.get('source', 'No source available'))
        # print("Similarity score:", similarities[0][most_similar_index])

    scores_docs_list.append(scores_doc)
    source_lists.append(source_list)

In [ ]:
# Documents ranking based on Cosine Similarity

from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
def get_sentence_embeddings(texts):
    return sentence_model.encode(texts)

def rank_documents(query, documents):
    query_embedding = get_sentence_embeddings([query])[0]
    doc_texts = [doc.page_content for doc in documents]
    doc_embeddings = get_sentence_embeddings(doc_texts)
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    ranked_indices = np.argsort(similarities)[::-1]
    return [documents[i] for i in ranked_indices]

query = "What specific characteristics must an asset have to be eligible as High-Quality Liquid Assets under OSFI guidelines?"
retrieved_docs = vectorstore.as_retriever(search_kwargs={"k": 5}).get_relevant_documents(query)
ranked_docs = rank_documents(query, retrieved_docs)

print(ranked_docs)

print([ranked_doc.metadata for ranked_doc in ranked_docs])

In [ ]:
# creating the Cosine Similarities report

import csv
import pandas as pd
import numpy as np


scores_list = []
for scores_doc in scores_docs_list:
  scores = list(scores_doc.values())
  scores_list.extend(scores)

cosine_sim_mean = np.mean(scores_list)
cosine_sim_median = np.median(scores_list)
cosine_sim_std = np.std(scores_list)
cosine_sim_min = np.min(scores_list)
cosine_sim_max = np.max(scores_list)

print(cosine_sim_median)
print(cosine_sim_mean)
print(cosine_sim_std)
print(cosine_sim_min)
print(cosine_sim_max)

prompt_list = [scores_doc.keys() for scores_doc in scores_docs_list]
prompt_list = [item for sublist in prompt_list for item in sublist]

df = pd.DataFrame(list(zip(prompt_list,scores_list)),columns=['Prompts','Cosine Similarity scores'])

df2 = pd.DataFrame({
    'Mean': [cosine_sim_mean],
    'Median':[cosine_sim_median],
    'Std': [cosine_sim_std],
    'Min': [cosine_sim_min],
    'Max': [cosine_sim_max]
})

df = pd.concat([df, df2], axis=1)

0.7554884015148109
0.7480915072048482
0.053519134096843377
0.5296140975080871
0.8505870150102594


In [ ]:
df.head(10)

,Prompts,Cosine Similarity scores,Mean,Median,Std,Min,Max
0,What specific characteristics must an asset ha...,0.766527,0.748092,0.755488,0.053519,0.529614,0.850587
1,What factors influence the determination of ca...,0.743690,NaN,NaN,NaN,NaN,NaN
2,How does OSFI assess the impact of market stre...,0.781822,NaN,NaN,NaN,NaN,NaN
3,How should institutions manage liquidity risk ...,0.759462,NaN,NaN,NaN,NaN,NaN
4,What does OSFI require regarding documentation...,0.702500,NaN,NaN,NaN,NaN,NaN
5,What are OSFI's expectations for financial ins...,0.810745,NaN,NaN,NaN,NaN,NaN
6,What are the reporting frequencies and deadlin...,0.705825,NaN,NaN,NaN,NaN,NaN
7,What is the ASF factor associated to: Total re...,0.818603,NaN,NaN,NaN,NaN,NaN
8,Stable non-maturity (demand) deposits and term...,0.723740,NaN,NaN,NaN,NaN,NaN
9,All less stable non-maturity deposits and term...,0.714615,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('/content/drive/MyDrive/chatbot/cosine_similarity_scores_GPT4o2.csv', index=False)